In [89]:
# !pip install pandas numpy
# !pip install nltk
# !pip install spacy
# !python -m spacy download en_core_web_sm
!pip install scikit-learn



  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------------ --------------------------- 3.5/11.1 MB 74.5 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.1 MB 73.6 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.1 MB 73.6 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.1 MB 73.6 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.1 MB 73.6 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.1 MB 73.6 MB/s eta 0:00:01
   ----------------------- ---------------- 6.5/11.1 MB 20.9 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.1 MB 19.2 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.1 MB 16.3 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.1 MB 15.2 MB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [113]:
import pandas as pd
import numpy as np

In [128]:
data = pd.read_csv('data/IMDB Dataset Lite.csv')
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,"Petter Mattei's ""Love in the Time of Money"" is...",positive
4,"Probably my all-time favorite movie, a story o...",positive
...,...,...
4995,"The Bermuda Triangle ,we are told in this wast...",negative
4996,"After watching this on the MST3K episode, I ha...",negative
4997,Boring movie. Poor plot. Poor actors. The movi...,negative
4998,"This is not a boring movie, the audience might...",negative


In [129]:
data.columns

Index(['review', 'sentiment'], dtype='object')

In [130]:
import re

def text_preprocessing(text_string):
    # Remove URLs
    text_string = re.sub('https://.*','',text_string)

    # Remove html tags
    text_string = re.sub('<[^>]+>','',text_string)

    # Replace any non alphanumeric character with a space
    text_string = re.sub(r'[^A-Za-z0-9]',' ',text_string)

    # Then, replace multiple spaces with a single space
    text_string = re.sub(r'\s+', ' ', text_string)

    return text_string

In [131]:
data['review'] = data['review'].apply(lambda x : text_preprocessing(x))

In [132]:
data['review'][0]

'One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked They are right as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence which set in right from the word GO Trust me this is not a show for the faint hearted or timid This show pulls no punches with regards to drugs sex or violence Its is hardcore in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary It focuses mainly on Emerald City an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda Em City is home to many Aryans Muslims gangstas Latinos Christians Italians Irish and more so scuffles death stares dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wouldn t dare Forget pretty

In [133]:
# STOP WORDS & TOKENIZE
import spacy

nlp = spacy.load('en_core_web_sm')

def remove_stop_words(text):
    doc = nlp(text)

    text_without_stop_words = [token.lemma_ for token in doc if not token.is_stop] # Remove stop words and lemmatize
    clean_text = ' '.join(text_without_stop_words)

    return clean_text

In [134]:
data['review'] = data['review'].apply(lambda x : remove_stop_words(x))
data['review']

0       reviewer mention watch 1 Oz episode ll hook ri...
1       wonderful little production filming technique ...
2       think wonderful way spend time hot summer week...
3       Petter Mattei s Love Time Money visually stunn...
4       probably time favorite movie story selflessnes...
                              ...                        
4995    Bermuda Triangle tell waste celluloid portal t...
4996    watch MST3 K episode wonder movie film borrow ...
4997    boring movie poor plot poor actor movie happen...
4998    boring movie audience stay chair fascinate sel...
4999    try sit bomb long ago disaster acting atrociou...
Name: review, Length: 5000, dtype: object

In [141]:
from sklearn.preprocessing import LabelEncoder

X = data['review']
y = data['sentiment']

In [142]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [143]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [168]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.fit_transform(X_test)

In [169]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB(alpha=1.0)

model.fit(X_train_vec, y_train)

MultinomialNB()

In [170]:
y_pred = model.predict(X_test_vec)

y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

In [147]:
from sklearn.metrics import classification_report, accuracy_score

print("Accuracy Score:", accuracy_score(y_test_labels, y_pred_labels))
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred_labels))

Accuracy Score: 0.504

Classification Report:
               precision    recall  f1-score   support

    negative       0.50      0.47      0.49       500
    positive       0.50      0.54      0.52       500

    accuracy                           0.50      1000
   macro avg       0.50      0.50      0.50      1000
weighted avg       0.50      0.50      0.50      1000



In [158]:
# TF-IDF vectorizer

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import re
import spacy

# nlp = spacy.load('en_core_web_sm')

def text_preprocessing(text_string):
    # Remove URLs
    text_string = re.sub('https://.*','',text_string)

    # Remove html tags
    text_string = re.sub('<[^>]+>','',text_string)

    # Replace any non alphanumeric character with a space
    text_string = re.sub(r'[^A-Za-z0-9]',' ',text_string)

    # Then, replace multiple spaces with a single space
    text_string = re.sub(r'\s+', ' ', text_string)

    return text_string

def remove_stop_words(text):
    doc = nlp(text)

    text_without_stop_words = [token.lemma_ for token in doc if not token.is_stop] # Remove stop words and lemmatize
    clean_text = ' '.join(text_without_stop_words)

    return clean_text

df = pd.read_csv('data/IMDB Dataset Lite.csv')
print(df.head())

df['review'] = df['review'].apply(lambda x : text_preprocessing(x))
print(df.head())

df['review'] = df['review'].apply(lambda x : remove_stop_words(x))
print(df.head())

# Split Data
X = df['review']  # Input text
y = df['sentiment']  # Target labels

# Encode Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into Training and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# TF-IDF Vectorizer: Fit on Training Data Only
vectorizer = TfidfVectorizer()  # Initialize vectorizer
X_train_vec = vectorizer.fit_transform(X_train)  # Fit and transform training data
X_test_vec = vectorizer.transform(X_test)  # Only transform test data

# Train Naive Bayes Classifier
model = MultinomialNB(alpha=1.0)  # Laplace smoothing
model.fit(X_train_vec, y_train)

# Predict on Test Data
y_pred = model.predict(X_test_vec)

# Decode Predicted and True Labels
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

# Evaluation
print("Accuracy Score:", accuracy_score(y_test_labels, y_pred_labels))
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred_labels))


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Petter Mattei's "Love in the Time of Money" is...  positive
4  Probably my all-time favorite movie, a story o...  positive
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production The filming tech...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Petter Mattei s Love in the Time of Money is a...  positive
4  Probably my all time favorite movie a story of...  positive
                                              review sentiment
0  reviewer mention watch 1 Oz episode ll hook ri...  positive
1  wonderful little production filming technique ...  positive
2  think wonderful way spend time hot summer week...  p

In [161]:
# Count vectorizer / Bag of words

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import re
import spacy

# nlp = spacy.load('en_core_web_sm')

def text_preprocessing(text_string):
    # Remove URLs
    text_string = re.sub('https://.*','',text_string)

    # Remove html tags
    text_string = re.sub('<[^>]+>','',text_string)

    # Replace any non alphanumeric character with a space
    text_string = re.sub(r'[^A-Za-z0-9]',' ',text_string)

    # Then, replace multiple spaces with a single space
    text_string = re.sub(r'\s+', ' ', text_string)

    return text_string

def remove_stop_words(text):
    doc = nlp(text)

    text_without_stop_words = [token.lemma_ for token in doc if not token.is_stop] # Remove stop words and lemmatize
    clean_text = ' '.join(text_without_stop_words)

    return clean_text

df = pd.read_csv('data/IMDB Dataset Lite.csv')
print(df.head())

df['review'] = df['review'].apply(lambda x : text_preprocessing(x))
print(df.head())

df['review'] = df['review'].apply(lambda x : remove_stop_words(x))
print(df.head())

# Split Data
X = df['review']  # Input text
y = df['sentiment']  # Target labels

# Encode Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into Training and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Bag-of-Words Vectorizer: Fit on Training Data Only
vectorizer = CountVectorizer()  # Changed to CountVectorizer
X_train_vec = vectorizer.fit_transform(X_train)  # Fit and transform training data
X_test_vec = vectorizer.transform(X_test)  # Only transform test data

# Train Naive Bayes Classifier
model = MultinomialNB(alpha=1.0)  # Laplace smoothing
model.fit(X_train_vec, y_train)

# Predict on Test Data
y_pred = model.predict(X_test_vec)

# Decode Predicted and True Labels
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

# Evaluation
print("Accuracy Score:", accuracy_score(y_test_labels, y_pred_labels))
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred_labels))


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Petter Mattei's "Love in the Time of Money" is...  positive
4  Probably my all-time favorite movie, a story o...  positive
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production The filming tech...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Petter Mattei s Love in the Time of Money is a...  positive
4  Probably my all time favorite movie a story of...  positive
                                              review sentiment
0  reviewer mention watch 1 Oz episode ll hook ri...  positive
1  wonderful little production filming technique ...  positive
2  think wonderful way spend time hot summer week...  p